In [1]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle 
import pandas as pd 
import numpy as np

In [ ]:
### Load the ANN Model, Scaler pickle file, One Hot Encoding pickle file
model = load_model('model.h5')

## Load the encoders and scalers
with open('onehot_encoder_geo.pkl','rb') as file:
    label_encoder_geo = pickle.load(file)
    
with open('label_encoder_gender.pkl','rb') as file:
    label_encoder_gender = pickle.load(file)
    
with open('scaler.pkl','rb') as file:
    scaler = pickle.load(file)

In [5]:
# Example input data
input_data = {
    'CreditScore' : 600,
    'Geography' : 'France',
    'Gender' : 'Male',
    'Age' : 40,
    'Tenure' : 3,
    'Balance' : 600000,
    'NumOfProducts' : 2,
    'HasCrCard' : 1,
    'IsActiveMember' : 1,
    'EstimatedSalary': 50000
}

In [ ]:
## One-hot encode 'Geography'
geo_encoded = label_encoder_geo.transform([[input_data['Geography']]]).toarray()  ##double bracket because we are giving list format data
geo_encoded_df = pd.DataFrame(geo_encoded, columns = label_encoder_geo.get_feature_names_out(['Geography']))
geo_encoded_df

/Users/vibhupratap/Documents/Projects/Churn_Modelling_Project-ANN-/venv/lib/python3.11/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [11]:
input_df = pd.DataFrame([input_data])
input_data

{'CreditScore': 600,
 'Geography': 'France',
 'Gender': 'Male',
 'Age': 40,
 'Tenure': 3,
 'Balance': 600000,
 'NumOfProducts': 2,
 'HasCrCard': 1,
 'IsActiveMember': 1,
 'EstimatedSalary': 50000}

In [13]:
## Encode categorical variables 
input_df['Gender'] = label_encoder_gender.transform(input_df['Gender'])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,1,40,3,600000,2,1,1,50000


In [14]:
## Concatination with One Hot encoded
input_df = pd.concat([input_df.drop("Geography", axis = 1), geo_encoded_df], axis = 1)
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,600000,2,1,1,50000,1.0,0.0,0.0


In [15]:
## Scaling the input data
input_scaled = scaler.transform(input_df)
input_scaled


array([[-0.53598516,  0.91324755,  0.10479359, -0.69539349,  8.38812313,
         0.80843615,  0.64920267,  0.97481699, -0.87683221,  1.00150113,
        -0.57946723, -0.57638802]])

In [16]:
## Prediction churn
predicition = model.predict(input_scaled)
predicition

1/1 [==============================] - 0s 132ms/step


array([[0.40640217]], dtype=float32)

In [19]:
predicition_proba = predicition[0][0]
predicition_proba

0.40640217

In [18]:
if predicition_proba > 0.5:
    print('The customer is likely to churn.')
else:
    print('The customer is not libely to churn.')

The customer is not libely to churn.
